# 读取图像，转成tensor

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from PIL import Image#################
from torchvision import transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt

image_name = 'lenaOriginal.png'
img = Image.open(image_name)
print(type(img))

img = np.array(img)
print(img.shape) #转numpy

print(img)
print(img.transpose((2, 0, 1)).shape)

print("-------------------")
img = torch.from_numpy(img.transpose((2, 0, 1)))
print(img.shape)
img = img.numpy().transpose((1, 2, 0))
print(img.shape)


<class 'PIL.PngImagePlugin.PngImageFile'>
(512, 512, 3)
[[[191 134  82]
  [191 134  82]
  [189 133  81]
  ...
  [133  83  52]
  [133  83  52]
  [133  83  52]]

 [[191 134  82]
  [191 134  82]
  [189 133  81]
  ...
  [133  83  52]
  [133  83  52]
  [133  83  52]]

 [[188 128  81]
  [188 128  81]
  [193 136  82]
  ...
  [136  83  49]
  [133  83  49]
  [133  83  49]]

 ...

 [[161 100  61]
  [161 100  61]
  [164 106  66]
  ...
  [ 39  21  21]
  [ 39  21  21]
  [ 39  21  21]]

 [[158  97  60]
  [158  97  60]
  [162 101  64]
  ...
  [ 38  20  21]
  [ 38  20  20]
  [ 38  20  20]]

 [[158  97  60]
  [158  97  60]
  [162 101  64]
  ...
  [ 38  20  21]
  [ 38  20  20]
  [ 38  20  20]]]
(3, 512, 512)
-------------------
torch.Size([3, 512, 512])
(512, 512, 3)


In [2]:
#!conda install torchvision -y
#!conda install -c conda-forge onnx onnxruntime -y

In [3]:
# 使用torchvision中自带的transforms
image = Image.open(image_name).convert('RGB')##############Image

image.show()
print(type(image))
image = transforms.ToTensor()(image) #Image -> Tensor
print(type(image), image.shape)
#==================================

image = image.unsqueeze(0)
print(image.shape)

image = image.squeeze(0)
print(image.shape)
image = transforms.ToPILImage()(image)# 
print(type(image))

totensor = transforms.ToTensor()
toPIL = transforms.ToPILImage()

<class 'PIL.Image.Image'>
<class 'torch.Tensor'> torch.Size([3, 512, 512])
torch.Size([1, 3, 512, 512])
torch.Size([3, 512, 512])
<class 'PIL.Image.Image'>


In [4]:
# 图像预处理 normalization
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

normalize = transforms.Normalize(
    mean=norm_mean,
    std=norm_std,
    inplace=False
)

#print(totensor(image))

image_nor = normalize(totensor(image))#...........................................
print(image_nor.mean(), image_nor.std())
print(image_nor.shape)

image_nor = toPIL(image_nor)
print(type(image_nor), image_nor)
image_nor.save("lena_normalized.jpg")
image_nor.show()

tensor(-0.4199) tensor(1.0752)
torch.Size([3, 512, 512])
<class 'PIL.Image.Image'> <PIL.Image.Image image mode=RGB size=512x512 at 0x1A7532E8830>


# 图像卷积

In [5]:
# 卷积
x = torch.tensor([[[[3., 0., 1., 2., 7., 4.], 
	              [1., 5., 8., 9., 3., 1.], 
	              [2., 7., 2., 5., 1., 3.], 
	              [0., 1., 3., 1., 7., 8.], 
	              [4., 2., 1., 6., 2., 8.],
	              [2., 4., 5., 2., 3., 9.]]]], requires_grad=False)


w = torch.tensor([[[[1., 0., -1.], 
				  [1., 0., -1.],
				  [1., 0., -1.]]]], requires_grad=True)


print(x.shape, w.shape)

out = F.conv2d(x, w, stride=1, padding=0) #n=6, f=3, o=4
print(out)

out1 = F.conv2d(x, w, stride=1, padding=1) #n+2p-f+1=6
print(out1)

out2 = F.conv2d(x, w, stride=2, padding=0) #(n-f)/s+1=2
print(out2)

torch.Size([1, 1, 6, 6]) torch.Size([1, 1, 3, 3])
tensor([[[[ -5.,  -4.,   0.,   8.],
          [-10.,  -2.,   2.,   3.],
          [  0.,  -2.,  -4.,  -7.],
          [ -3.,  -2.,  -3., -16.]]]], grad_fn=<ConvolutionBackward0>)
tensor([[[[ -5.,  -5.,  -6.,  -1.,   6.,  10.],
          [-12.,  -5.,  -4.,   0.,   8.,  11.],
          [-13., -10.,  -2.,   2.,   3.,  11.],
          [-10.,   0.,  -2.,  -4.,  -7.,  10.],
          [ -7.,  -3.,  -2.,  -3., -16.,  12.],
          [ -6.,   0.,  -2.,   1.,  -9.,   5.]]]],
       grad_fn=<ConvolutionBackward0>)
tensor([[[[-5.,  0.],
          [ 0., -4.]]]], grad_fn=<ConvolutionBackward0>)


In [6]:
# 灰度图卷积：边缘检测
grayimage = image.convert('L') #
grayimage = totensor(grayimage).unsqueeze(0)
print(grayimage.shape)

w = torch.tensor([[[[1., 0., -1.], 
				  [2., 0., -2.],
				  [1., 0., -1.]]]])

convv = F.conv2d(grayimage, w, stride=1, padding=0)#

convv = convv.squeeze(0)#4d->3d

print(convv.shape)

convv = toPIL(convv)
convv.show()

torch.Size([1, 1, 512, 512])
torch.Size([1, 510, 510])


In [7]:
# padding
print(grayimage.shape)
convpad = F.conv2d(grayimage, w, stride=1, padding=10)
convpad = convpad.squeeze(0)
print(convpad.shape)
convpad = toPIL(convpad)
convpad.show()

torch.Size([1, 1, 512, 512])
torch.Size([1, 530, 530])


In [8]:
# 步长
convstride = F.conv2d(grayimage, w, stride=2, padding=0)
convstride = convstride.squeeze(0)
print(convstride.shape)
convstride = toPIL(convstride)
convstride.show()

torch.Size([1, 255, 255])


In [9]:
# 水平线检测
w2 = torch.tensor([[[[1., 2., 1.], 
				  [0., 0., 0.],
				  [-1., -2., -1.]]]])
convh = F.conv2d(grayimage, w2, stride=1, padding=0)
convh = convh.squeeze(0)
convh = toPIL(convh)
convh.show()

In [17]:
# 多通道卷积（RGB）
image = Image.open(image_name).convert('RGB')##############Image

image = totensor(image).unsqueeze(0)# 3d --> 4d

print(image.shape)

#yellow kernel
w = torch.tensor([[[[1., 0., -1.], 
				  [2., 0., -2.],
				  [1., 0., -1.]]]])

print("before, w.shape= ", w.shape)

w = w.repeat(1,3,1,1)

print("after, w.shape= ", w.shape)

convv = F.conv2d(image, w, stride=1, padding=0)
convv = convv.squeeze(0)
print(convv.shape)
convv = toPIL(convv)
convv.show()

print(w)

torch.Size([1, 3, 512, 512])
before, w.shape=  torch.Size([1, 1, 3, 3])
after, w.shape=  torch.Size([1, 3, 3, 3])
torch.Size([1, 510, 510])
tensor([[[[ 1.,  0., -1.],
          [ 2.,  0., -2.],
          [ 1.,  0., -1.]],

         [[ 1.,  0., -1.],
          [ 2.,  0., -2.],
          [ 1.,  0., -1.]],

         [[ 1.,  0., -1.],
          [ 2.,  0., -2.],
          [ 1.,  0., -1.]]]])


In [11]:
convpad = F.conv2d(image, w, stride=1, padding=10)
convpad = convpad.squeeze(0)
convpad = toPIL(convpad)
convpad.show()

In [12]:
convstride = F.conv2d(image, w, stride=2, padding=0)
convstride = convstride.squeeze(0)
convstride = toPIL(convstride)
convstride.show()

In [14]:
# 多卷积核
w2 = torch.tensor([[[[1., 2., 1.], 
				  [0., 0., 0.],
				  [-1., -2., -1.]]]])

print(w.shape)
w2 = w2.repeat(1,3,1,1)
print("w2.shape=", w2.shape)

print("----------------------------------")
w3 = torch.cat((w,w2),0)
print("first cat, w3.shape=", w3.shape)
w3 = torch.cat((w3,w2),0)
print("secode cat, w3.shape=", w3.shape)

convh = F.conv2d(image, w3, stride=1, padding=0)

print("convh.shape = ", convh.shape)
convh = convh.squeeze(0)
convh = toPIL(convh)
convh.show()

torch.Size([1, 3, 3, 3])
w2.shape= torch.Size([1, 3, 3, 3])
----------------------------------
first cat, w3.shape= torch.Size([2, 3, 3, 3])
secode cat, w3.shape= torch.Size([3, 3, 3, 3])
convh.shape =  torch.Size([1, 3, 510, 510])
